In [1]:
import xtrack as xt
import xobjects as xo
import xcoll as xc
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors

In [2]:
line = xt.Line.from_json('injection_lines/injection_thin_approx_ap_with_deferred_expressions.json')

#Set cavitiy voltages correctly
line['acl.31735'].frequency = 800e6
line['acl.31735'].voltage = 2 * 0.18e6
line['acl.31735'].lag = 180

line['actcse.31632'].frequency = 200e6
line['actcse.31632'].voltage = 4.5e6
line['actcse.31632'].lag = 180

tw = line.twiss()

Loading line from dict:   0%|          | 0/31362 [00:00<?, ?it/s]

Done loading line from dict.           


In [3]:
num_turns = 1000
num_particles = 1000

nemitt_x = 3.5e-6
nemitt_y = 3.5e-6

In [4]:
idx_tidvg = np.where(tw.name=='tidvg.51872')[0][0]
ap_tidvg_x = 78.6e-3/2
ap_tidvg_y = 40.8e-3/2

sigma_tidvg_x, sigma_tidvg_y = np.sqrt(tw.betx[idx_tidvg]*nemitt_x/tw.gamma0), np.sqrt(tw.bety[idx_tidvg]*nemitt_y/tw.gamma0)
gap_x, gap_y = ap_tidvg_x/sigma_tidvg_x, ap_tidvg_y/sigma_tidvg_y

tidvg_1 = xc.EverestCollimator(length=2.5, material = xc.materials.Carbon, gap_R = -gap_y, angle = 90, side = 'right') #it is graphite but apparently that dos not exist (except Molybdenium graphite)
tidvg_2 = xc.EverestCollimator(length=1.0, material = xc.materials.Aluminium, gap_R = -gap_y, angle = 90, side = 'right')
tidvg_3 = xc.EverestCollimator(length=0.5, material = xc.materials.Copper, gap_R = -gap_y, angle = 90, side = 'right')
tidvg_4 = xc.EverestCollimator(length=0.3, material = xc.materials.Tungsten, gap_R = -gap_y, angle = 90, side = 'right')

In [ ]:
line.discard_tracker()
center_drift = 0.35
begin_tidvg = tw.s[idx_tidvg] + center_drift
line.collimators.install(names=['tidvg.51872..1_C', 'tidvg.51872..2_Al', 'tidvg.51872..3_Cu', 'tidvg.51872..4_W'], elements=[tidvg_1, tidvg_2, tidvg_3, tidvg_4], at_s=[begin_tidvg, begin_tidvg+2.5, begin_tidvg+3.5, begin_tidvg+4])
#Getting rid of the np.str_ type
max_length =max(map(len, line.element_names))
line.element_names = np.array(line.element_names, dtype=f'<U{max_length}').tolist()

Slicing line:   0%|          | 0/30212 [00:00<?, ?it/s]

In [ ]:
tw= line.twiss()